# Example: Diet Model

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy 

### Step 1: Import amplpy

In [1]:
from __future__ import print_function
from amplpy import AMPL, DataFrame
import pandas as pd
import amplpy

### Step 2: Create an AMPL object

In [2]:
ampl = AMPL()

### Step 3: Define the model

In [3]:
ampl.eval('''
set FOOD ;
set NUTR ;
# Parameters
param cost { FOOD } > 0;
param f_min { FOOD } >= 0;
param f_max {j in FOOD } >= f_min [j];
param n_min { NUTR } >= 0;
param n_max {i in NUTR } >= n_min [i];
param amt {NUTR , FOOD } >= 0;
# Variables
var Buy {j in FOOD } >= f_min [j], <= f_max [j];
# Objective
minimize total_cost : sum {j in FOOD } cost [j] * Buy[j];
# Contraints
subject to diet {i in NUTR }:
    n_min [i] <= sum {j in FOOD } amt[i,j] * Buy[j] <= n_max [i];
''')

### Step 4: Define the initial data

In [4]:
foods = ['BEEF', 'CHK', 'FISH', 'HAM', 'MCH', 'MTL', 'SPG', 'TUR']
nutrients = ['A', 'C', 'B1', 'B2', 'NA', 'CAL']

ampl.getSet('FOOD').setValues(foods)
ampl.getSet('NUTR').setValues(nutrients)

#### Define data using an amplpy DataFrame

In [5]:
ampl.setData(DataFrame(
    index=[('FOOD', foods)], 
    columns=[
        ('cost', [3.59, 2.59, 2.29, 2.89, 1.89, 1.99, 1.99, 2.49]), 
        ('f_min', [2, 2, 2, 2, 2, 2, 2, 2]), 
        ('f_max', [10, 10, 10, 10, 10, 10, 10, 10])
    ]
))

#### Define data using a Pandas DataFrame

In [6]:
df = pd.DataFrame({
    'n_min': [700, 700, 700, 700, 0, 16000], 
    'n_max': [20000, 20000, 20000, 20000, 50000, 24000]
}, 
    index=nutrients
)
ampl.setData(DataFrame.fromPandas(df))

In [7]:
amounts = [
    [ 60,    8,   8,  40,   15,  70,   25,   60],
    [ 20,    0,  10,  40,   35,  30,   50,   20],
    [ 10,   20,  15,  35,   15,  15,   25,   15],
    [ 15,   20,  10,  10,   15,  15,   15,   10],
    [928, 2180, 945, 278, 1182, 896, 1329, 1397],
    [295,  770, 440, 430,  315, 400,  379,  450]
]
df = DataFrame(('NUTR', 'FOOD'), 'amt')
df.setValues({
    (nutrient, food): amounts[i][j]
    for i, nutrient in enumerate(nutrients)
    for j, food in enumerate(foods)
})
ampl.setData(df)

### Step 5: Solve the model

In [8]:
ampl.solve()

MINOS 5.51: optimal solution found.
13 iterations, objective 119.9897589


### Step 6: Create a Pandas DataFrame with the values of the variable 'Buy'

In [9]:
ampl.getVariable('Buy').getValues().toPandas()

,Buy.val
BEEF,5.226933
CHK,2.000000
FISH,2.000000
HAM,10.000000
MCH,10.000000
MTL,10.000000
SPG,9.439734
TUR,2.000000


### Step 7: Display the objective value

In [10]:
totalcost = ampl.getObjective('total_cost')
print("Objective is:", totalcost.value())

Objective is: 119.989758936


### Step 8: Increase the costs of beef and ham

In [11]:
cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

Increased costs of beef and ham.


### Step 9: Solve the model with the new costs

In [12]:
ampl.solve()

MINOS 5.51: optimal solution found.
0 iterations, objective 144.0120033


### Step 10: Display the new objective value

In [13]:
print("New objective value:", totalcost.value())

New objective value: 144.012003325


### Step 11: Display the values of the variable 'Buy' using Bokeh

In [14]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

rows = [tuple(row) for row in ampl.getVariable('Buy').getValues()]
factors = [index for index, value in rows]
x = [value for index, value in rows]

dot = figure(
    title="Categorical Dot Plot", tools='', toolbar_location=None,
    y_range=factors, x_range=[0,12]
)

dot.segment(0, factors, x, factors, line_width=2, line_color='green')
dot.circle(x, factors, size=15, fill_color='orange', line_color='green', line_width=3)
show(dot)

Loading BokehJS ...